# DIT821 Software Engineering for AI systems

#### Exam  2020-08-17

Reinforcement Learning - Value Iteration Algorithm

Enter here your first and last name and your e-mail adress as registered in Canvas.

    * Name Shab Pompeiano, e-mail: guspomsh@student.gu.se

In this assignement, you will implement the value iteration algorithm on a given gridworld environment (MDP).

In order to run this lab you have to install the [gym environment](https://gym.openai.com/docs/) first.
In order to install it, simply run in the terminal:
```
    pip3 install gym
```
or
```
    pip install gym
```
It is assumed that you have installed numpy already, otherwise install it (e.g. as with gym).

Please make sure that you have the lib folder in the same directory of this file. 

Write the code  and comments according to the requirement, and run it.

Donwload the file as a notenook file (.ipynb) and submit it to canvas.

In [1]:
# Make sure that everything here is imported correctly
import numpy as np
import sys
from lib.envs.gridworld import GridworldEnv

#### Generating the gridworld
The next command generated a 4x4 grid and saves the MDP in the variable `env`.

The gridwold looks like this:

    T  o  o  o
    o  x  o  o
    o  o  o  o
    o  o  o  T

Where x is the position of the agent and T are the terminal states.

The agent, at each state, can take 4 actions: `UP=0, RIGHT=1, DOWN=2, LEFT=3`

Both states and action are represented with integer values.

Actions going off the edge leave you in your current state.

The agent receives a reward of -1 at each step until you reach a terminal state. If the state is terminal every action will give the agent a reward of 1.

***`env.P`*** represents the transition probabilities of the environment.

***`env.P[s][a]`***: given a state `s` and an action `a`, it returns a list of transition tuples `(prob, next_state, reward, done)` where `prob` is the probability associated to the next state (`next_state`), `reward` is the reward associated to the transition and `done` is a boolean indicating if `next_state` is terminal.

***`env.nS`*** is a number of states in the environment (16 in this example). 

***`env.nA`*** is a number of actions in the environment (4 in this example).

For more information, feel free to consult the `gridworld.py` file inside the `lib/envs/` folder.


In [2]:
# Load the 4x4 gridworld environment inside the variable 'env'
""" env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
    env.nS is a number of states in the environment. 
    env.nA is a number of actions in the environment."""
env = GridworldEnv()

# Fixing the discount factor
discount_factor=1.0

####  Getting the q-values (values for each action from a given state): 

The following function will help you in the development of your algorithm.

It takes a state `state` and the numpy Vector `V` of size `env.nS` containing the current values for each state.

It return the values of `state` after step look-ahead for ***all*** the actions. Since we have four actions avilable at each state it returns 4 values of `state`, one for each action.

It is the core of the value function algorithm. Check out the slides.


In [3]:
def get_values_one_step(state, V):
    """
    Helper function to calculate the value for all action in a given state.

    Args:
        state: The state to consider (int)
        env:   OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
        V:     The vector of currect values to use as an estimator, Vector of length env.nS
        discount_factor: discount factor

    Returns:
        A vector of length env.nA containing the expected value of each action.
    """
    # Vector to return     
    A = np.zeros(env.nA)
    
    # *** INSERT CODE HERE ***
    # TIP: Use env.P[state][action] to get the list of (probability, next_state, reward, done).
    
    for i in range (env.nA):
        for prob, next_state, reward, done in env.P[state][i]:
            A[i] = A[i] + prob * (discount_factor * V[next_state] + reward)
            
    return A

#### In the next cell, you should  complete the value iteration function using the `get_q_values_for_state` function.


In [4]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        theta: We stop evaluation once our value function change is less than theta for all states.        
    Returns:
        V: numpy Vector containing the optimal value for each state of env.
    """

    
    # Initialize to zero the numpy Vector containing env.nS values, one for each state of the MDP.
    V = np.zeros(env.nS)
    
    while True:
        # Stopping condition
        delta = 0
        
        # Update each state...
        for s in range(env.nS):
            
            # keep old state value in memory for calculating the delta            
            old_s = V[s]
            
            # Update the value function, i.e. the vector V
            # *** INSERT CODE HERE ***
            #  TIP: Do a one-step lookahead from state s to find the best action

            A = get_values_one_step(s, V)
            
            best_action = np.argmax(A) 
            
            q = 0
            for prob, next_state, reward, done in env.P[s][best_action]:
                q = q + prob * (discount_factor * V[next_state] + reward)
            
            V[s] = q
            
            
            # Calculate delta across all states seen so far (for stopping condition)
            delta = max(delta, np.abs(V[s] - old_s))
            
        # Check if we can stop 
        if delta < theta:
            break
    
    return V

#### Optimal Policy (BONUS)
Once we have all the optimal values for each state we can compute the optimal Policy
It is not mandatory to complete the following part of the lab as well.

In [5]:
def compute_optimal_policy(V):
    """
    Computes the optival policy
    
    Args:
        V: numpy Vector containing the optimal values for each state in env
        
    Returns:
        policy: some data structure returning the best action for each state
    """
    # ***INSERT CODE HERE***
    # TIP: Create a deterministic policy using the optimal value function
    
    policy = []
    
    for s in V:
        max_q = env.P[s][0]
        for i in env.nA:
            if max_q <env.P[s][i]:
                max_q = env.P[s][i]
                best_action = i
        policy.push(best_action)


    
    
    return policy

### Check if your solution is correct
Run the following code to check if the value functions that your functions generates are correct

In [6]:
v = value_iteration(env)

# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
try:
    np.testing.assert_array_almost_equal(v, expected_v, decimal=2)
    print("CONGRATS! THE SOLUTION YOU HAVE PROVIDED IS CORRECT!")
except AssertionError as e :
    print("SOLUTION NOT CORRECT")
    raise e

CONGRATS! THE SOLUTION YOU HAVE PROVIDED IS CORRECT!


## Submit the solution

When you completed the excercise, download (form File menu) this file as a jupyter Notebook file (.ipynb) and uplaod this file in the CANVAS 

By writing down my name I declare that we have done the assignement myself:

* First Name: Shab  Last Name: Pompeiano
